In [17]:
import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.neighbors import NearestNeighbors
from scipy import stats
from utils import *

import librosa
import librosa.display

import os
import multiprocessing

from tqdm import tqdm

data_path = 'data'
fma_small_path = 'data/fma_small'
fma_meta_path = 'data/fma_metadata'

tracks = fma_load(f'{data_path}/tracks_small.csv')
train = tracks['set', 'split'] == 'training'
test = tracks['set', 'split'] == 'test'

In [10]:
def columns():
    feature_sizes = dict(chroma_stft=12,
                         chroma_cqt=12,
                         chroma_cens=12,
                         tonnetz=6,
                         mfcc=20,
                         rmse=1,
                         zcr=1,
                         spectral_centroid=1,
                         spectral_bandwidth=1,
                         spectral_contrast=7,
                         spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()

def compute_features(params):
    [tid, duration] = params

    print(f'computing features for: {duration}s of {tid}.\n')

    features = pd.Series(index=columns(), dtype=np.float32, name=f'{tid}-{duration}')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)


    filepath = f'{fma_small_path}/{tid[0:3]}/{tid}.mp3'
    x, sr = librosa.load(filepath, sr=None, mono=True, duration=duration)  # kaiser_fast

    f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
    feature_stats('zcr', f)

    cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                n_bins=7*12, tuning=None))
    assert cqt.shape[0] == 7 * 12
    assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

    f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
    feature_stats('chroma_cqt', f)
    f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
    feature_stats('chroma_cens', f)
    f = librosa.feature.tonnetz(chroma=f)
    feature_stats('tonnetz', f)

    del cqt
    stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
    assert stft.shape[0] == 1 + 2048 // 2
    assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
    del x

    f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
    feature_stats('chroma_stft', f)

    f = librosa.feature.rms(S=stft)
    feature_stats('rmse', f)

    f = librosa.feature.spectral_centroid(S=stft)
    feature_stats('spectral_centroid', f)
    f = librosa.feature.spectral_bandwidth(S=stft)
    feature_stats('spectral_bandwidth', f)
    f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
    feature_stats('spectral_contrast', f)
    f = librosa.feature.spectral_rolloff(S=stft)
    feature_stats('spectral_rolloff', f)

    mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
    del stft
    f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
    feature_stats('mfcc', f)

    return features

In [11]:
param = ['000002', 2]
f = compute_features(param)

computing features for: 2s of 000002.



In [12]:
# tracks = tracks.loc[test].head(50)
len(tracks)

8000

In [13]:
# features = pd.DataFrame(index=tracks.index,
#                         columns=columns(), dtype=np.float32)
features = pd.DataFrame(columns=columns(), dtype=np.float32)
# More than usable CPUs to be CPU bound, not I/O bound. Beware memory.
nb_workers = int(1.5 * os.cpu_count())

# Longest is ~11,000 seconds. Limit processes to avoid memory errors.
# all are lower than 600
# table = ((5000, 1), (3000, 3), (2000, 5), (1000, 10), (0, nb_workers))
# for duration, nb_workers in table:
print('Working with {} processes.'.format(nb_workers))

# tids = tracks[tracks['track', 'duration'] >= duration].index
# tracks.drop(tids, axis=0, inplace=True)

tids = ['{:06d}'.format(i) for i in tracks.index]
frame_size = 1
whole_duration = 30
durations = list(range(frame_size, whole_duration+1, frame_size))
durations = [30]
params = [ [i, d] for i in tids for d in durations]
print(params)
print(len(params))

692', 30], ['130693', 30], ['130702', 30], ['130709', 30], ['130710', 30], ['130711', 30], ['130752', 30], ['130758', 30], ['130920', 30], ['130921', 30], ['130922', 30], ['130923', 30], ['130927', 30], ['130929', 30], ['130930', 30], ['130931', 30], ['130932', 30], ['130933', 30], ['130934', 30], ['130937', 30], ['130940', 30], ['130944', 30], ['130945', 30], ['130948', 30], ['130950', 30], ['130951', 30], ['130952', 30], ['130953', 30], ['130954', 30], ['130955', 30], ['130956', 30], ['130963', 30], ['130964', 30], ['130986', 30], ['130988', 30], ['130989', 30], ['130990', 30], ['130991', 30], ['130992', 30], ['130993', 30], ['131016', 30], ['131019', 30], ['131020', 30], ['131021', 30], ['131024', 30], ['131166', 30], ['131292', 30], ['131323', 30], ['131324', 30], ['131325', 30], ['131326', 30], ['131327', 30], ['131385', 30], ['131410', 30], ['131422', 30], ['131425', 30], ['131426', 30], ['131436', 30], ['131439', 30], ['131444', 30], ['131446', 30], ['131448', 30], ['131449', 30

In [14]:
pool = multiprocessing.Pool(nb_workers)

# it = pool.imap_unordered(compute_features, tids)
it = pool.imap_unordered(compute_features, params)

computing features for: 30s of 142097.

computing features for: 30s of 142098.

computing features for: 30s of 142129.

computing features for: 30s of 142128.

computing features for: 30s of 142132.

computing features for: 30s of 142133.

computing features for: 30s of 142358.

computing features for: 30s of 142359.

computing features for: 30s of 142360.

computing features for: 30s of 142361.

computing features for: 30s of 142362.

computing features for: 30s of 142381.

computing features for: 30s of 142402.

computing features for: 30s of 142418.

computing features for: 30s of 142433.

computing features for: 30s of 142511.

computing features for: 30s of 142517.
computing features for: 30s of 142516.


computing features for: 30s of 142519.

computing features for: 30s of 142528.

computing features for: 30s of 142529.

computing features for: 30s of 142530.

computing features for: 30s of 142531.

computing features for: 30s of 142532.

computing features for: 30s of 142533.



In [15]:
for i, row in enumerate(tqdm(it, total=len(params))):
    features.loc[row.name] = row
    if i % 1000 == 0:
        features.to_csv('data/duration_features.csv')

features.to_csv('data/duration_features.csv')

 56%|█████▌    | 4453/8000 [33:03<26:19,  2.25it/s]


NoBackendError: 

In [8]:
duration_features = fma_load('data/duration_features.csv') 
duration_features.loc[duration_features.index == '000182-30']['mfcc']

statistics  kurtosis                                                   \
number            01        02       03        04        05        06   
000182-30   0.428096 -0.464112  0.36862 -0.495273 -0.478796 -0.269317   

statistics                                          ...       std            \
number            07        08        09        10  ...        11        12   
000182-30   0.512002  0.101145 -0.246554  0.146798  ...  7.284407  6.693648   

statistics                                                             \
number            13        14        15       16        17        18   
000182-30   6.591286  6.149924  5.781596  6.17767  6.159792  6.500729   

statistics                     
number            19       20  
000182-30   6.105147  6.29175  

[1 rows x 140 columns]

In [ ]:
tracks = tracks.loc[test].head(400)
# len(tracks)

# features = pd.DataFrame(index=tracks.index,
#                         columns=columns(), dtype=np.float32)
features = pd.DataFrame(columns=columns(), dtype=np.float32)
# More than usable CPUs to be CPU bound, not I/O bound. Beware memory.
nb_workers = int(1.5 * os.cpu_count())

# Longest is ~11,000 seconds. Limit processes to avoid memory errors.
# all are lower than 600
# table = ((5000, 1), (3000, 3), (2000, 5), (1000, 10), (0, nb_workers))
# for duration, nb_workers in table:
print('Working with {} processes.'.format(nb_workers))

# tids = tracks[tracks['track', 'duration'] >= duration].index
# tracks.drop(tids, axis=0, inplace=True)

tids = ['{:06d}'.format(i) for i in tracks.index]
frame_size = 1
whole_duration = 30
durations = list(range(frame_size, whole_duration+1, frame_size))
# durations = [30]
params = [ [i, d] for i in tids for d in durations]
print(params)
print(len(params))

pool = multiprocessing.Pool(nb_workers)

# it = pool.imap_unordered(compute_features, tids)
it = pool.imap_unordered(compute_features, params)

for i, row in enumerate(tqdm(it, total=len(params))):
    features.loc[row.name] = row
    if i % 1000 == 0:
        features.to_csv('data/duration_features2.csv')

features.to_csv('data/duration_features2.csv')